In [1]:
import torch
from torch import nn
from torchvision.models import resnet34


In [4]:
encoded = torch.load('quantized33564.pt')
print(encoded.size()) # = torch.Size([1, 33564, 576])
encoded1 = encoded[0]
print(encoded1.size()) # = torch.Size([33564, 576])

torch.Size([1, 33564, 576])
torch.Size([33564, 576])


In [24]:
class ResNet34Decoder(nn.Module):
    def __init__(self):
        super(ResNet34Decoder, self).__init__()
        self.resnet = resnet34(pretrained=True)
        self.resnet.conv1 = nn.Identity()  # remove the first layer
        self.upsample1 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.upsample2 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.upsample3 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.upsample4 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

    def forward(self, x):
        x = self.resnet(x)
        x = self.upsample1(x)
        x = self.upsample2(x)
        x = self.upsample3(x)
        x = self.upsample4(x)
        return x

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.conv = nn.Conv2d(33564, 64, kernel_size=1)  # replace the fully connected layer with a convolutional layer
        # self.conv2 = nn.Conv2d(128, 64, kernel_size=1)  # add a new convolutional layer to reduce the number of feature maps to 64
        self.resnet_decoder = ResNet34Decoder()

    def forward(self, x):
        x = x.view(1, x.size(0), 24, 24)  # reshape the input for the convolutional layer
        x = self.conv(x)  # pass through convolutional layer
        # x = self.conv2(x)  # pass through the second convolutional layer
        x = self.resnet_decoder(x)  # pass through ResNet decoder
        x = x.view(x.size(0), -1)  # flatten the output
        x = x.view(x.size(0), 361, 720)  # reshape to the desired size
        return x
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available, else use CPU
model = Decoder().to(device)  # Move model to the device

decoded = model(encoded1).to(device)
print(decoded.size())


NotImplementedError: Input Error: Only 3D, 4D and 5D input Tensors supported (got 2D) for the modes: nearest | linear | bilinear | bicubic | trilinear | area | nearest-exact (got bilinear)

In [31]:
encoded = torch.load('quantized33564.pt')
print(encoded.size()) # = torch.Size([1, 33564, 576])
encoded1 = encoded[0]
print(encoded1.size()) # = torch.Size([33564, 576])

class ResNet34Decoder(nn.Module):
    def __init__(self):
        super(ResNet34Decoder, self).__init__()
        self.resnet = resnet34(pretrained=True)
        self.resnet.conv1 = nn.Identity()  # remove the first layer
        self.upsample1 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.upsample2 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.upsample3 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.upsample4 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

    def forward(self, x):
        x = self.resnet(x)
        x = self.upsample1(x)
        x = self.upsample2(x)
        x = self.upsample3(x)
        x = self.upsample4(x)
        return x

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.conv = nn.Conv1d(576, 64, kernel_size=1)  # use Conv1d instead of Conv2d
        self.resnet_decoder = ResNet34Decoder()

    def forward(self, x):
        x = x.transpose(1, 2)  # swap the second and third dimensions
        x = self.conv(x)  # pass through convolutional layer
        print('after conv',x.size())
        x = x.unsqueeze(-1)  # add two extra dimensions
        print('after unsqueeze',x.size())
        x = self.resnet_decoder(x)  # pass through ResNet decoder
        x = x.squeeze(-1)  # remove the extra dimensions
        x = x.view(x.size(0), -1)  # flatten the output
        x = x.view(x.size(0), 361, 720)  # reshape to the desired size
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available, else use CPU
model = Decoder().to(device)  # Move model to the device

decoded = model(encoded).to(device)
print(decoded.size())


torch.Size([1, 33564, 576])
torch.Size([33564, 576])
after conv torch.Size([1, 64, 33564])
after unsqueeze torch.Size([1, 64, 33564, 1])


NotImplementedError: Input Error: Only 3D, 4D and 5D input Tensors supported (got 2D) for the modes: nearest | linear | bilinear | bicubic | trilinear | area | nearest-exact (got bilinear)

In [32]:
import torch
import torch.nn as nn

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.conv1 = nn.ConvTranspose1d(576, 64, 5, stride=2, padding=1)
        self.conv2 = nn.ConvTranspose1d(64, 32, 5, stride=2, padding=1)
        self.conv3 = nn.ConvTranspose1d(32, 16, 5, stride=2, padding=1)
        self.conv4 = nn.ConvTranspose2d(16, 8, 5, stride=2, padding=1)
        self.conv5 = nn.ConvTranspose2d(8, 4, 5, stride=2, padding=1)
        self.conv6 = nn.ConvTranspose2d(4, 1, 5, stride=2, padding=1)

    def forward(self, x):
        x = x.view(1, 576, -1)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.view(1, 16, int(x.size(2)**0.5), -1)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available, else use CPU
model = Decoder().to(device)  # Move model to the device

encoded = torch.load('quantized33564.pt')
encoded1 = encoded[0]

decoded = model(encoded1).to(device)
print(decoded.shape)  # should be [1, 1, 361, 720]


RuntimeError: shape '[1, 16, 518, -1]' is invalid for input of size 4296304

In [36]:
import torch
import torch.nn as nn

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.conv1 = nn.ConvTranspose1d(576, 128, 5, stride=2, padding=1)
        self.conv2 = nn.ConvTranspose1d(128, 64, 5, stride=2, padding=1)
        self.conv3 = nn.ConvTranspose1d(64, 32, 5, stride=2, padding=1)

        self.conv4 = nn.ConvTranspose2d(32, 16, 5, stride=2, padding=1)
        self.conv5 = nn.ConvTranspose2d(16, 8, 5, stride=2, padding=1)
        self.conv6 = nn.ConvTranspose2d(8, 1, 5, stride=2, padding=1)

    def forward(self, x):
        x = x.view(x.size(0), 576, -1)  # adjust to match the input shape
        print(x.size())
        x = self.conv1(x)
        print(x.size())
        x = self.conv2(x)
        print(x.size())
        x = self.conv3(x)
        print(x.size())
        x = x.view(x.size(0), 32, 361, -1)  # adjust to match the desired shape
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available, else use CPU
model = Decoder().to(device)  # Move model to the device

encoded = torch.load('quantized33564.pt')
encoded1 = encoded[0]

decoded = model(encoded1).to(device)
print(decoded.shape)  # should be [1, 1, 361, 720]


torch.Size([33564, 576, 1])
torch.Size([33564, 128, 3])
torch.Size([33564, 64, 7])
torch.Size([33564, 32, 15])


RuntimeError: shape '[33564, 32, 361, -1]' is invalid for input of size 16110720

In [42]:
import torch
import torch.nn as nn

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.conv1 = nn.ConvTranspose1d(576, 128, 5, stride=2, padding=1)
        self.conv2 = nn.ConvTranspose1d(128, 64, 5, stride=2, padding=1)
        self.conv3 = nn.ConvTranspose1d(64, 32, 5, stride=2, padding=1)

        self.conv4 = nn.ConvTranspose1d(32, 16, 5, stride=2, padding=1)
        self.conv5 = nn.ConvTranspose1d(16, 8, 5, stride=2, padding=1)
        self.conv6 = nn.ConvTranspose1d(8, 1, 5, stride=2, padding=1)

    def forward(self, x):
        print(x.size())
        x = x.view(x.size(0), 576, -1)  # adjust to match the input shape
        print(x.size())
        x = self.conv1(x)
        print(x.size())
        x = self.conv2(x)
        print(x.size())
        x = self.conv3(x)
        print(x.size())
        # x = x.view(x.size(0), 32, 361, -1)  # adjust to match the desired shape
        x = self.conv4(x)
        print(x.size())
        x = self.conv5(x)
        print(x.size())
        x = self.conv6(x)
        print(x.size())
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available, else use CPU
model = Decoder().to(device)  # Move model to the device

encoded = torch.load('quantized33564.pt')
encoded1 = encoded[0]

decoded = model(encoded1).to(device)
# print(decoded.shape)  # should be [1, 1, 361, 720]


torch.Size([33564, 576])
torch.Size([33564, 576, 1])
torch.Size([33564, 128, 3])
torch.Size([33564, 64, 7])
torch.Size([33564, 32, 15])
torch.Size([33564, 16, 31])
torch.Size([33564, 8, 63])
torch.Size([33564, 1, 127])
torch.Size([33564, 1, 127])


In [43]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        # Encoder
        self.conv1 = nn.ConvTranspose1d(576, 128, 5, stride=2, padding=1)
        self.conv2 = nn.ConvTranspose1d(128, 64, 5, stride=2, padding=1)
        self.conv3 = nn.ConvTranspose1d(64, 32, 5, stride=2, padding=1)
        self.conv4 = nn.ConvTranspose1d(32, 16, 5, stride=2, padding=1)
        self.conv5 = nn.ConvTranspose1d(16, 8, 5, stride=2, padding=1)
        self.conv6 = nn.ConvTranspose1d(8, 1, 5, stride=2, padding=1)

        self.enc1 = nn.Linear(in_features=33564*127, out_features=1024)
        #OutOfMemoryError: CUDA out of memory. Tried to allocate 16.26 GiB. GPU 0 has a total capacty of 12.00 GiB of which 6.86 GiB is free. Including non-PyTorch memory, this process has 17179869184.00 GiB memory in use. Of the allocated memory 3.90 GiB is allocated by PyTorch, and 163.20 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF
        self.enc2 = nn.Linear(in_features=1024, out_features=256)
        self.enc3 = nn.Linear(in_features=256, out_features=64)
        
        # Decoder
        self.dec1 = nn.Linear(in_features=64, out_features=256)
        self.dec2 = nn.Linear(in_features=256, out_features=1024)
        self.dec3 = nn.Linear(in_features=1024, out_features=361*720)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        # Encode
        x = F.relu(self.enc1(x))
        x = F.relu(self.enc2(x))
        x = F.relu(self.enc3(x))
        
        # Decode
        x = F.relu(self.dec1(x))
        x = F.relu(self.dec2(x))
        x = self.dec3(x)
        
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available, else use CPU
model = Autoencoder().to(device)  # Move model to the device

encoded = torch.load('quantized33564.pt')
encoded1 = encoded[0]

decoded = model(encoded).to(device)
print(decoded.shape)  # should be [1, 1, 361, 720]

OutOfMemoryError: CUDA out of memory. Tried to allocate 16.26 GiB. GPU 0 has a total capacty of 12.00 GiB of which 6.86 GiB is free. Including non-PyTorch memory, this process has 17179869184.00 GiB memory in use. Of the allocated memory 3.90 GiB is allocated by PyTorch, and 163.20 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [58]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.conv1 = nn.ConvTranspose1d(576, 128, 5, stride=2, padding=1) #输入为(batch,dim,length)
        self.conv2 = nn.ConvTranspose1d(128, 64, 5, stride=2, padding=1)
        self.conv3 = nn.ConvTranspose1d(64, 32, 5, stride=2, padding=1)

        self.conv4 = nn.ConvTranspose1d(32, 16, 5, stride=2, padding=1)
        self.conv5 = nn.ConvTranspose1d(16, 8, 5, stride=2, padding=1)
        self.conv6 = nn.ConvTranspose1d(8, 1, 5, stride=2, padding=1)

    def forward(self, x):
        pad_size = 36000 - x.size(1)
        x = F.pad(x, (0, 0, 0, pad_size))
        print(x.size())

        x = x.view(x.size(0), 576, -1)  # adjust to match the input shape, 1,576,33564
        print(x.size())
        
        x = self.conv1(x)
        print(x.size())
        x = self.conv2(x)
        print(x.size())
        x = self.conv3(x)
        print(x.size())
        x = self.conv4(x)
        print(x.size())
        x = self.conv5(x)
        print(x.size())
        x = self.conv6(x)
        print(x.size())
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available, else use CPU
model = Decoder().to(device)  # Move model to the device

encoded = torch.load('quantized33564.pt') #input size is torch.size([1,33564,576])
# encoded1 = encoded[0]

decoded = model(encoded).to(device)
# print(decoded.shape)  # should be [1, 1, 361, 720], but not


torch.Size([1, 36000, 576])
torch.Size([1, 576, 36000])
torch.Size([1, 128, 72001])
torch.Size([1, 64, 144003])
torch.Size([1, 32, 288007])
torch.Size([1, 16, 576015])
torch.Size([1, 8, 1152031])
torch.Size([1, 1, 2304063])


In [62]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.conv1 = nn.ConvTranspose1d(576, 128, 5, stride=2, padding=1) #输入为(batch,dim,length)
        self.pool1 = nn.MaxPool1d(2)  # Add a max pooling layer after the first convolutional layer
        self.conv2 = nn.ConvTranspose1d(128, 64, 5, stride=2, padding=1)
        self.pool2 = nn.MaxPool1d(2)  # Add a max pooling layer after the second convolutional layer
        self.conv3 = nn.ConvTranspose1d(64, 32, 5, stride=2, padding=1)
        self.pool3 = nn.MaxPool1d(2)  # Add a max pooling layer after the third convolutional layer

        self.conv4 = nn.ConvTranspose1d(32, 16, 5, stride=2, padding=1)
        self.pool4 = nn.MaxPool1d(2)  # Add a max pooling layer after the fourth convolutional layer
        self.conv5 = nn.ConvTranspose1d(16, 8, 5, stride=2, padding=1)
        self.pool5 = nn.MaxPool1d(2)  # Add a max pooling layer after the fifth convolutional layer
        self.conv6 = nn.ConvTranspose1d(8, 1, 5, stride=2, padding=1)
        self.pool6 = nn.MaxPool1d(2)  # Add a max pooling layer after the fifth convolutional layer

        # Encoder2
        self.enc1 = nn.Linear(in_features=36000, out_features=1024)
        # self.enc2 = nn.Linear(in_features=1024, out_features=256)
        # self.enc3 = nn.Linear(in_features=256, out_features=64)
        
        # # Decoder2
        # self.dec1 = nn.Linear(in_features=64, out_features=256)
        # self.dec2 = nn.Linear(in_features=256, out_features=1024)
        # self.dec3 = nn.Linear(in_features=1024, out_features=361*720)

    def forward(self, x):
        pad_size = 36000 - x.size(1)
        x = F.pad(x, (0, 0, 0, pad_size))
        print(x.size())

        x = x.view(x.size(0), 576, -1)  # adjust to match the input shape, 1,576,33564
        print(x.size())
        
        x = self.conv1(x)
        x = self.pool1(x)  # Apply max pooling after the first convolutional layer
        print(x.size())
        x = self.conv2(x)
        x = self.pool2(x)  # Apply max pooling after the second convolutional layer
        print(x.size())
        x = self.conv3(x)
        x = self.pool3(x)  # Apply max pooling after the third convolutional layer
        print(x.size())
        # x = x.view(x.size(0), 32, 361, -1)  # adjust to match the desired shape
        x = self.conv4(x)
        x = self.pool4(x)  # Apply max pooling after the fourth convolutional layer
        print(x.size())
        x = self.conv5(x)
        x = self.pool5(x)  # Apply max pooling after the fifth convolutional layer
        print(x.size())
        x = self.conv6(x)
        x = self.pool6(x)
        print(x.size())
        x=x.squeeze(1)
        print(x.size())

        x=F.relu(self.enc1(x))
        print(x.size())
        return x
    

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available, else use CPU
model = Decoder().to(device)  # Move model to the device

encoded = torch.load('quantized33564.pt') #input size is torch.size([1,33564,576])
# encoded1 = encoded[0]

decoded = model(encoded).to(device)
# print(decoded.shape)  # should be [1, 1, 361, 720], but not


torch.Size([1, 36000, 576])
torch.Size([1, 576, 36000])
torch.Size([1, 128, 36000])
torch.Size([1, 64, 36000])
torch.Size([1, 32, 36000])
torch.Size([1, 16, 36000])
torch.Size([1, 8, 36000])
torch.Size([1, 1, 36000])
torch.Size([1, 36000])
torch.Size([1, 1024])


In [47]:
print(decoded)

tensor([[[0.2148, 0.3765, 0.3731,  ..., 0.3775, 0.4003, 0.3978]]],
       device='cuda:0', grad_fn=<SqueezeBackward1>)


In [52]:
print(decoded.squeeze(1).size()) #squeeze(dim)移除指定维度（如果该维度长1），不指定的话就是移除所有为1的维度，不为1的话指定了白指定

torch.Size([1, 36000])


In [64]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.conv1 = nn.ConvTranspose1d(576, 128, 5, stride=2, padding=1) #输入为(batch,dim,length)
        self.pool1 = nn.MaxPool1d(2)  # Add a max pooling layer after the first convolutional layer
        self.conv2 = nn.ConvTranspose1d(128, 64, 5, stride=2, padding=1)
        self.pool2 = nn.MaxPool1d(2)  # Add a max pooling layer after the second convolutional layer
        self.conv3 = nn.ConvTranspose1d(64, 32, 5, stride=2, padding=1)
        self.pool3 = nn.MaxPool1d(2)  # Add a max pooling layer after the third convolutional layer

        self.conv4 = nn.ConvTranspose1d(32, 16, 5, stride=2, padding=1)
        self.pool4 = nn.MaxPool1d(2)  # Add a max pooling layer after the fourth convolutional layer
        self.conv5 = nn.ConvTranspose1d(16, 8, 5, stride=2, padding=1)
        self.pool5 = nn.MaxPool1d(2)  # Add a max pooling layer after the fifth convolutional layer
        self.conv6 = nn.ConvTranspose1d(8, 1, 5, stride=2, padding=1)
        self.pool6 = nn.MaxPool1d(2)  # Add a max pooling layer after the fifth convolutional layer

        # Decoder
        self.dec1 = nn.Linear(in_features=36000, out_features=2**15)
        self.dec2 = nn.Linear(in_features=2**15, out_features=2**16)
        self.dec3 = nn.Linear(in_features=2**16, out_features=2**17)
        self.dec4 = nn.Linear(in_features=2**17, out_features=361*720)
        
        # # Decoder2
        # self.dec1 = nn.Linear(in_features=64, out_features=256)
        # self.dec2 = nn.Linear(in_features=256, out_features=1024)
        # self.dec3 = nn.Linear(in_features=1024, out_features=361*720)

    def forward(self, x):
        pad_size = 36000 - x.size(1)
        x = F.pad(x, (0, 0, 0, pad_size))
        print(x.size())

        x = x.view(x.size(0), 576, -1)  # adjust to match the input shape, 1,576,33564
        print(x.size())
        
        x = self.conv1(x)
        x = self.pool1(x)  # Apply max pooling after the first convolutional layer
        print(x.size())
        x = self.conv2(x)
        x = self.pool2(x)  # Apply max pooling after the second convolutional layer
        print(x.size())
        x = self.conv3(x)
        x = self.pool3(x)  # Apply max pooling after the third convolutional layer
        print(x.size())
        x = self.conv4(x)
        x = self.pool4(x)  # Apply max pooling after the fourth convolutional layer
        print(x.size())
        x = self.conv5(x)
        x = self.pool5(x)  # Apply max pooling after the fifth convolutional layer
        print(x.size())
        x = self.conv6(x)
        x = self.pool6(x)
        print(x.size())
        x=x.squeeze(1)
        print(x.size())

        x=F.relu(self.dec1(x))
        print(x.size())
        x=F.relu(self.dec2(x))
        print(x.size())
        x=F.relu(self.dec3(x))
        print(x.size())
        x=F.relu(self.dec4(x))
        print(x.size())
        
        return x
    

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available, else use CPU
model = Decoder().to(device)  # Move model to the device

encoded = torch.load('quantized33564.pt') #input size is torch.size([1,33564,576])
# encoded1 = encoded[0]

decoded = model(encoded).to(device)
# print(decoded.shape)  # should be [1, 1, 361, 720], but not


: 

# 逆天 居然成功了。
但是感觉最慌的就是1024到361*720的升维映射，最后可以在这儿优化。

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.conv1 = nn.ConvTranspose1d(576, 128, 5, stride=2, padding=1) #输入为(batch,dim,length)
        self.pool1 = nn.MaxPool1d(2)  # Add a max pooling layer after the first convolutional layer
        self.conv2 = nn.ConvTranspose1d(128, 64, 5, stride=2, padding=1)
        self.pool2 = nn.MaxPool1d(2)  # Add a max pooling layer after the second convolutional layer
        self.conv3 = nn.ConvTranspose1d(64, 32, 5, stride=2, padding=1)
        self.pool3 = nn.MaxPool1d(2)  # Add a max pooling layer after the third convolutional layer

        self.conv4 = nn.ConvTranspose1d(32, 16, 5, stride=2, padding=1)
        self.pool4 = nn.MaxPool1d(2)  # Add a max pooling layer after the fourth convolutional layer
        self.conv5 = nn.ConvTranspose1d(16, 8, 5, stride=2, padding=1)
        self.pool5 = nn.MaxPool1d(2)  # Add a max pooling layer after the fifth convolutional layer
        self.conv6 = nn.ConvTranspose1d(8, 1, 5, stride=2, padding=1)
        self.pool6 = nn.MaxPool1d(2)  # Add a max pooling layer after the fifth convolutional layer

        # Encoder2
        self.fc1 = nn.Linear(in_features=36000*8, out_features=1024)
        self.fc2 = nn.Linear(in_features=1024, out_features=361*720)

    def forward(self, x):
        pad_size = 36000 - x.size(1)
        x = F.pad(x, (0, 0, 0, pad_size))
        print(x.size())

        x = x.view(x.size(0), 576, -1)  # adjust to match the input shape, 1,576,33564
        print(x.size())
        
        x = self.conv1(x)
        x = self.pool1(x)  # Apply max pooling after the first convolutional layer
        print(x.size())
        x = self.conv2(x)
        x = self.pool2(x)  # Apply max pooling after the second convolutional layer
        print(x.size())
        x = self.conv3(x)
        x = self.pool3(x)  # Apply max pooling after the third convolutional layer
        print(x.size())
        # x = x.view(x.size(0), 32, 361, -1)  # adjust to match the desired shape
        x = self.conv4(x)
        x = self.pool4(x)  # Apply max pooling after the fourth convolutional layer
        print(x.size())
        x = self.conv5(x)
        x = self.pool5(x)  # Apply max pooling after the fifth convolutional layer
        print(x.size())
        # x = self.conv6(x)
        # x = self.pool6(x)
        # print(x.size())
        
        x=x.view(1,-1)
        print(x.size())

        x=F.relu(self.fc1(x))
        print(x.size())
        x=F.relu(self.fc2(x))
        print(x.size())

        x=x.view(1,361,-1)
        print(x.size())
        return x
    

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available, else use CPU
model = Decoder().to(device)  # Move model to the device

encoded = torch.load('quantized33564.pt') #input size is torch.size([1,33564,576])
# encoded1 = encoded[0]

decoded = model(encoded).to(device)
# print(decoded.shape)  # should be [1, 1, 361, 720], but not


torch.Size([1, 36000, 576])
torch.Size([1, 576, 36000])
torch.Size([1, 128, 36000])
torch.Size([1, 64, 36000])
torch.Size([1, 32, 36000])
torch.Size([1, 16, 36000])
torch.Size([1, 8, 36000])
torch.Size([1, 288000])
torch.Size([1, 1024])
torch.Size([1, 259920])
torch.Size([1, 361, 720])
